# Lession 4 - 文本推理

In [11]:
import openai
import os

os.environ["http_proxy"] = "http://127.0.0.1:<端口号>"
os.environ["https_proxy"] = "http://127.0.0.1:<端口号>"

openai.api_key  = 'sk-'

def get_completion(prompt, model="gpt-3.5-turbo"):
    '''
    prompt: 对应的提示
    model: 调用的模型，默认为 gpt-3.5-turbo(ChatGPT)，有内测资格的用户可以选择 gpt-4
    '''
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # 模型输出的温度系数，控制输出的随机程度
    )
    # 调用 OpenAI 的 ChatCompletion 接口
    return response.choices[0].message["content"]

## 1 情感类型识别

对于从产品评论以及新闻文章中推断出情感和主题的任务来实，其实质可看做是一个模型接受文本输入并对该输入进行分析的过程，该过程可能涉及到标签提取、实体提取、文本情感理解等多方面。
在传统机器学习任务中，如果想从一段文本中提取正面/负面情感，需要收集标签数据，通过标签数据训练模型以及在云端部署模型并进行推理服务。这无疑是一个非常繁琐的过程，并且对于每个不同的任务，例如情感分析、实体提取等，都需要获取不同的标注数据去重新训练模型并进行部署。

这种针对每种任务去分别训练模型的方式直到LLM的出现才有所变化，在LLM中，无需大量工作，只需要通过一个Prompt即可令LLM产生不错的结果，从而有效加快了应用程序开发的速度。此外还可以借助一个模型以及API来实现多种不同人分五，无需再去弄清如何训练和部署多种应对不同任务的模型。

### 1.1 正/负向情感分类

以一盏台灯的购买评论为例，我们可以通过构建Prompt来实现对该评论的情感分类

In [12]:
text = """
我需要一盏漂亮的卧室灯，这款灯具有额外的储物功能，价格也不算太高。\
我很快就收到了它。在运输过程中，我们的灯绳断了，但是公司很乐意寄送了一个新的。\
几天后就收到了。这款灯很容易组装。我发现少了一个零件，于是联系了他们的客服，他们很快就给我寄来了缺失的零件！\
在我看来，Lumina 是一家非常关心顾客和产品的优秀公司！
"""

In [13]:
prompt = f"""
以下用三个反引号分隔的产品评论的情感是什么？
评论文本: ```{text}```
"""
response = get_completion(prompt)
print(response)

积极的情感。


### 1.2 多种情感类型识别

In [14]:
prompt = f"""
识别以下评论的作者表达的情感。包含不超过五个项目。将答案格式化为以逗号分隔的单词列表。
评论文本: ```{text}```
"""
response = get_completion(prompt)
print(response)

满意,感激,赞扬,信任,愉快


In [16]:
text_2 = """
物流选择的实在是太差劲了！比蜗牛还慢，脖子都等长了！耽误了我给女友过生日，女友生气了，差点跟我分手，我倒霉透顶，给你差评!"""

prompt = f"""
以下评论的作者是否表达了愤怒？评论用三个反引号分隔。给出是或否的答案。
评论文本: ```{text_2}```
"""
response = get_completion(prompt)
print(response)

是


### 1.3 信息提取 (Information Retrieval)与信息抽取(Information Extraction)

在NLP任务中，**信息抽取（Information Extraction）** 与 **信息提取（Information Retrieval）** 是两个不同的概念：
1. 信息抽取：从非结构化或半结构化的文本中识别并提取出结构化的实体、关系以及事件等其他内容，以便于后续分析和使用。常见任务有：实体识别、关系收取以及事件抽取等。
2. 信息提取：根据用户的查询或需求从大规模文本数据库中检索出相关文档或信息，实质上是一个通过比对用户的查询和文本数据，进行文本匹配和排序的过程，即在文本数据中检索出最相关的信息呈现给用户。常见任务包括文本搜索、文档分类等。

因此，信息抽取更注重结构化的信息，信息提取着重于从文本数据中找出相关信息。

#### 1.3.1 信息提取

关于信息提取部分，具体可见 **L3-Summarizing** 中的1.3节 关键信息提取部分。

In [17]:
text = """
罗辑与比尔·希恩斯在进入冬眠大约200年之后被唤醒，这已经是一个完全不同的世界。地球在过去的两个世纪里曾经为了备战的考虑，工业与科技的投入使得社会被完全破坏，环境条例被废除，导致生态灾难，地球表面大面积沙漠化，人口因饥饿与疾病而锐减了数十亿，而这段黑暗的历史则被称谓大低谷。同时，各国则挖掘了庞大的地下城市作为三体舰队达到时的防御掩体。而在经历了大低谷之后的人类回归到发展自身社会上来，着重于民生与经济的投入，使得在罗辑苏醒的年代，人类社会已经达到十分富足与和平的近乎乌托邦的世界。科学上，尽管有智子的封锁，地球的技术仍然取得了巨大进步，太空舰队已拥有数以千计的战舰。有了强大的舰队，许多人在讨论未来与三体人的和平谈判。被唤醒之后，罗辑立即受邀参加了联合国就面壁者计划的最后一次会议，借由大低谷后数十年来的应用技术爆炸发展，世界已经开始相信人类已经掌握了高于三体舰队的军力，并且对末日之战人类的胜利信心十足，在此由各国向罗辑与希恩斯宣布了面壁计划的结束。罗辑终于恢复了平民的身份，在放下面壁者的重任后开始努力融入这个新的地球社会，因为在此时，社会结构、价值观、伦理观相较于二十一世纪初都已经发生了翻天覆地的变化。同时罗辑也碰见了同样冬眠了上百年的大史。由于未来的城市系统依靠着联网的机器而运作，二人却不断地莫名碰到各种针对罗辑的袭击——一种由ETO最后的残党制造的已经隐藏了数十年的计算机病毒——Killer 5.2，试图利用它能控制的各种机器来谋杀罗辑。为了逃过追杀，罗辑与大史求助于警察，并计划搬往地面上由冬眠者组成的更为“原始”的社区居住。
"""

prompt = f"""
你的任务是针对三个反引号之间的小说内容中罗辑的经历进行概述，最多50个字。
小说内容：```{text}```
"""

response = get_completion(prompt)
print(response)

罗辑苏醒后参加联合国面壁者计划的最后一次会议，得知人类已经掌握高于三体舰队的军力，面壁计划结束。他开始融入新的地球社会，但遭遇计算机病毒Killer 5.2的袭击，与大史一起逃亡并搬往更为“原始”的社区居住。


#### 1.3.2 信息抽取

在信息抽取任务中，可以通过LLM从非结构化的文本数据中抽取出关键的信息，并转化为结构化的格式。例如在下列JD的职业要求中，抽取出结构化的关键信息

In [21]:
schema = {
    'JD岗位要求': [
        '学历要求: ',
        '专业要求: ',
        '工作年限要求: ',
        '编程语言: ',
        '加分项: '
    ]
}

text = """职位要求：1、本科以上学历。2、电子信息或软件工程专业。3、1-3年工作经验。4、熟练掌握java或者c++语言。5、有相关项目经验优先"""

prompt = f"""
需要你协助完成信息抽取任务，当我给你一个引号中的JD职位要求时，帮我抽取出句子中三元组，并按照三个反引号中的json的格式输出，上述句子中没有的信息用['该JD未要求']来表示，多个值之间用','分隔。
Json格式：```{schema}```
JD职位要求："{text}"
"""

response = get_completion(prompt)
print(response)

```{'JD岗位要求': ['学历要求: 本科以上', '专业要求: 电子信息或软件工程', '工作年限要求: 1-3年', '编程语言: java或者c++', '加分项: 有相关项目经验优先']}```


# 2. 文本内容推理

尝试使用ChatGPT推导出《三体Ⅱ：黑暗森林》中的黑暗森林法则

In [30]:
Axiom = """
1. 生存是文明的第一需要。
2. 文明不断增长和扩张，但宇宙中的物质总量保持不变。
"""

Supplement_1 = """
猜疑链：双方无法判断对方是否为善意文明。
一个文明不能判断另一个文明是善文明还是恶文明
一个文明不能判断另一个文明认为本文明是善文明还是恶文明
一个文明不能判断另一个文明是否会对本文明发起攻击
一个文明无法判断另一个文明对自己是善意或恶意的
一个文明无法判断另一个文明认为自己是善意或恶意的
一个文明无法判断另一个文明判断自己对它是善意或恶意的
"""

Supplement_2 = """
技术爆炸：文明进步的速度和加速度不见得是一致的，弱小的文明很可能在短时间内超越强大的文明。
"""

hypothesis = """
1. 宇宙里的文明只有最低程度的善意。
2. 交流一定会暴露文明的位置。
3. 文明之间的竞争残酷，高等文明一定会选择打击
4. 不同智慧物种间难以沟通，且成本高昂
5. 任何文明都有很长的寿命。
6. 高等文明的大杀器存在并成本低廉
"""

premise = """强大的高等文明A遇到了弱小的落后文明B，为了高等文明A的生存"""

In [31]:
prompt = f"""
需要你帮我基于给出的三个反引号中的公理，引号中的补充以及方括号中的假设内容，完成一个推理过程
你的推理过程必须严格遵守给出的公理，补充内容以及假设内容，不可违反。
公理内容：```{Axiom}```;
补充内容：
    补充1："{Supplement_1}";
    补充2："{Supplement_2}"
假设内容：[{hypothesis}]
现在给出前提：{premise} \
请在该前提下，完成后续推导并给出推导过程
注意：你的推理过程必须严格遵守给出的公理，补充内容以及假设内容，不可违反。
"""

response = get_completion(prompt)
print(response)

推导过程如下：

1. 根据公理1，生存是文明的第一需要，因此高等文明A会优先考虑自身的生存和发展。
2. 根据补充1，高等文明A无法判断落后文明B是否为善意文明，因此无法确定是否可以与其和平共处。
3. 根据补充2，落后文明B可能在短时间内超越高等文明A，因此高等文明A需要采取措施防止落后文明B的崛起。
4. 根据假设3，高等文明A会选择打击落后文明B，以确保自身的生存和发展。
5. 根据假设2，交流会暴露文明的位置，因此高等文明A不会与落后文明B进行交流，而是采取隐蔽的方式进行攻击。
6. 根据假设6，高等文明A拥有成本低廉的大杀器，可以轻易地摧毁落后文明B。
7. 综上所述，高等文明A会选择打击落后文明B，以确保自身的生存和发展。
